### Метрики классификации.


In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [29]:
data = pd.read_csv("/home/UBkarima/phase_1_hw/1_Среда_3/microsoft_malware_detection.csv")

In [30]:
data.head()

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,Platform_windows8,Processor_x64,Processor_x86,Census_DeviceFamily_Windows.Server,Census_OSArchitecture_arm64,Census_OSArchitecture_x86,Census_GenuineStateName_IS_GENUINE,Census_GenuineStateName_OFFLINE,Census_GenuineStateName_UNKNOWN,HasDetections
0,0.552153,0.530941,0.587952,7.0,0,53447.0,1.0,1.0,1,29,...,0,1,0,0,0,0,1,0,0,0
1,0.448751,0.528931,0.484434,7.0,0,53447.0,1.0,1.0,1,93,...,0,1,0,0,0,0,0,1,0,0
2,0.552153,0.530941,0.557522,7.0,0,53447.0,1.0,1.0,1,86,...,0,1,0,0,0,0,1,0,0,0
3,0.552153,0.530941,0.482759,7.0,0,53447.0,1.0,1.0,1,97,...,0,1,0,0,0,0,1,0,0,0
4,0.493175,0.530941,0.505721,7.0,0,53447.0,1.0,1.0,1,164,...,0,1,0,0,0,0,1,0,0,0


Перед вами пердобработанный датасет от Microsoft. столбец  `HasDetections` - ваш таргет. Который отражает по имеющимся данным, данная программа является вирусом или нет. Датасет имеет дисбаланс классов. Хороша ли метрика `accuracy`?

1. Проверьте пропорцию классов

In [34]:
class_counts = data['HasDetections'].value_counts(normalize=True)  # Доли классов
print(class_counts)

# Визуализация распределения классов
plt.figure(figsize=(6, 4))
class_counts.plot(kind='bar', color=['blue', 'red'])
plt.xlabel('Класс (HasDetections)')
plt.ylabel('Доля')
plt.title('Распределение классов')
plt.xticks(rotation=0)
plt.show()


2. Сделайте scaling данных

Произведем процедуру отложенной выборки:

3. Создайте разделение на тренировочную и валидационную выборку, постройте модель Логистической регрессии, попробуйте настроить коэффициент регуляризации(С) с помощью `optuna`

In [35]:
from sklearn.preprocessing import StandardScaler
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns
data_scaled = data.copy()
scaler = StandardScaler()
data_scaled[numerical_features] = scaler.fit_transform(data[numerical_features])
print(data_scaled.head())

4.  Постройте __confusion matrix__ на тестовой выборки. Отобразите ее в виде DataFrame

|          | y_real = 1  | y_real = 0  |
|----------|------------|------------|
|  y_pred = 1  |     TP     |     FP     |
|  y_pred = 0  |     FN     |     TN     |  

Напоминаю:

* Positive/Negative отражает то, к какому классу определила объект наша модель, в нашем случае нулевой класс соответствует Negative (нет вируса), а единичный класс - Positive (есть вирус).

* True/False просто показывает совпало ли предсказание модели с реальным классом объекта, поэтому на главной диагонали стоят True, а на побочной False.

In [ ]:
X = data[numerical_features].drop(columns=['HasDetections'], errors='ignore')   
y = data['HasDetections']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-4, 10)  
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return f1_score(y_val, y_pred)  


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)



5. Сравните результат с методом `confusion_matrix` в `sklearn.metrics`. Обратите внимание на тот факт, что в sklearn матрица транспонирована относительно нашей

In [20]:
from sklearn.metrics import confusion_matrix
y_pred = final_model.predict(X_val)
cm = confusion_matrix(y_val, y_pred)

tn, fp, fn, tp = cm.ravel()

print("Confusion Matrix:")
print(cm)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


array([[24726,   293],
       [ 4442,   527]])

6. Посчитайте `accuracy`, `precision`, `recall`

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Получаем предсказания модели
y_pred = final_model.predict(X_val)

# Вычисляем метрики с sklearn
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)

# Вывод результатов
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Альтернативный расчет через confusion_matrix
cm = confusion_matrix(y_val, y_pred)
tn, fp, fn, tp = cm.ravel()

accuracy_manual = (tp + tn) / (tp + tn + fp + fn)
precision_manual = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_manual = tp / (tp + fn) if (tp + fn) > 0 else 0

print("\nРассчитано вручную:")
print(f"Accuracy: {accuracy_manual:.4f}")
print(f"Precision: {precision_manual:.4f}")
print(f"Recall: {recall_manual:.4f}")

In [8]:
from sklearn.metrics import recall_score, precision_score

### Your code is here

7. Какая метрика более точно описывает происходящее? Хороший ли результат мы получили

In [9]:
### Your code is here



8. Допустим, руководитель требует, чтобы наш классификатор в 60% случаев правильно предсказывал наличие вируса на компьютере, чтобы не беспокоить клиентов из-за ложноположительных результатов. \
Как решить эту задачу? =>
Надо подобрать порог, который максимизирует `recall` при условии, что `precision > 0.6`.

$$
(recall | precision > 0.6) \to \max_{thres}
$$

Для этого возможно построить так называемую `precision_recall_curve`, которая поможет нам посмотреть при разных трешхолдах соотношения этих двух метрик и вырать оптимальное

In [35]:
from sklearn.metrics import precision_recall_curve

### Строим всевозможные комбинации precision, recall, threshold

precision, recall, thres = precision_recall_curve(y_test, pred_probs[:, 1])

### Находим такую тройку, при которой recall максимален, а precision > 0.6

ind = np.where(recall == recall[precision > 0.6].max())

print(f"Precison:\t {precision[ind][-1]}")
print(f"Recall: \t {recall[ind][-1]}")
print(f"Threshold:\t {thres[ind][-1]}")

Precison:	 0.6003717472118959
Recall: 	 0.1300060374320789
Threshold:	 0.45766970448177774


Теперь чтобы соответствовать установленным условиям, мы будем замерять вероятности для объектов и сравнивать и с найденным `Threshold`, который решает нашу задачу

9. На примере того, что сделано  решите задачу, где заказчик требует, чтобы наш классификатор находил хотя бы 20% компьютеров с вирусом, среди всех действительно зараженных компьютеров.
$$
(precision | recall > 0.2) \to \max_{thres}
$$

In [10]:
### Your code is here



10. Одним из самых распространенных показателей качества является **F-мера**, которая обобщает все вышерассмотренные метрики. Более того, в ней можно регулировать параметр $\beta^2$, который отвечает за предпочтение между precision и recall. 
$$
F = \dfrac{(1 + \beta^2) P \cdot R}{\beta^2 P + R}
$$
* При $\beta^2 = 1$ и precision, и recall нам важны в равной степени.
* При $\beta^2 > 1$ предпочтение отдается recall. Похоже на решение задачи, где мы фиксируем приемлемое для нас значение precision и максимизируем recall.
* При $0 < \beta^2 < 1$ предпочтение отдается precision. Похоже на решение задачи, где мы фиксируем приемлемое для нас значение recall и максимизируем precision.

Найдите порог, который максимизирует:
1) F-меру с $\beta^2 = 1$,
2) F-меру с $\beta^2 = 0.05$,
3) F-меру с $\beta^2 = 4$.\
Для всех трех случаев выведите precision, recall, threshold и F-меру. 
Нарисуйте линии уровня и точку, которая максимизирует F-меру по аналогии с предыдущим пунктом.

In [13]:
### Повторите для F-меры при \beta^2=1
### Выведите ее вместе с precision/recall/threshold
### Your code is here


In [14]:
### Повторите для F-меры при \beta^2=0.05
### Выведите ее вместе с precision/recall/threshold
### Your code is here






In [15]:
### Повторите для F-меры при \beta^2=4
### Выведите ее вместе с precision/recall/threshold
### Your code is here



11. Постройте roc-auc.  
Пример описан снизу

In [ ]:
# from sklearn.metrics import roc_curve

# fpr, tpr, thresholds = roc_curve(y_valid, y_predict_proba)

# # И нарисуем ROC-кривую

# from sklearn.metrics import RocCurveDisplay

# RocCurveDisplay(fpr=fpr, tpr=tpr).plot()